In [1]:
import awscli
import selenium
import boto3
import os
import s3fs 
import pandas as pd
import time
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")

In [2]:
repo = os.path.dirname(os.path.abspath(''))

In [3]:
browser = webdriver.Chrome(os.path.join(repo, "ResourceDatasets", "chromedriver.exe"))

#enter the url path that needs to be accessed by webdriver
browser.get('https://data.cccnewyork.org/data/table/66/median-incomes#66/107/62/a/a')
time.sleep(5)
#identify xpath of location to select element
table = browser.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/div[3]/div/table")
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])

defined_columns = ['location','all_households','families','families_with_children','families_without_children']

In [4]:
df_median_BOROUGHS = pd.DataFrame(df, columns=df[1])[4:9].reset_index(drop=True)
df_median_BOROUGHS.columns = defined_columns
df_median_BOROUGHS

,location,all_households,families,families_with_children,families_without_children
0,Bronx,"$41,432","$50,835","$41,129","$61,248"
1,Brooklyn,"$66,937","$74,422","$66,936","$79,400"
2,Manhattan,"$93,651","$126,690","$140,841","$121,669"
3,Queens,"$73,696","$82,534","$75,501","$86,501"
4,Staten Island,"$89,821","$105,438","$104,641","$106,015"


In [5]:
df_median_DISTRICTS = pd.DataFrame(df, columns=df[1])[10:69].reset_index(drop=True)
df_median_DISTRICTS['Rank / Location'] = [df_median_DISTRICTS['Rank / Location'][i][:-6] for i in range(len(df_median_DISTRICTS))]
df_median_DISTRICTS.columns = defined_columns
df_median_DISTRICTS.head(5)

,location,all_households,families,families_with_children,families_without_children
0,Astoria,"$79,180","$88,662","$75,972","$92,755"
1,Battery Park/Tribeca,"$162,092","$230,219","$250,001","$204,568"
2,Bay Ridge,"$76,569","$89,345","$91,935","$87,713"
3,Bayside,"$92,682","$108,371","$120,639","$102,316"
4,Bedford Park,"$41,336","$47,784","$40,863","$56,260"


In [6]:
df_median_DISTRICTS_updated = df_median_DISTRICTS.copy()
df_median_DISTRICTS_updated['location'] = df_median_DISTRICTS_updated.location.str.upper().\
                                                        replace('ELMHURST/CORONA','ELMHURST').\
                                                        replace('JAMAICA/ST. ALBANS','JAMAICA').\
                                                        replace('REGO PARK/FOREST HILLS','REGO PARK').\
                                                        replace('FRESH MEADOWS/BRIARWOOD','FRESH MEADOWS').\
                                                        replace('RIDGEWOOD/GLENDALE','RIDGEWOOD').\
                                                        replace('SUNNYSIDE/WOODSIDE','Sunnyside')


new_CORONA = df_median_DISTRICTS_updated[df_median_DISTRICTS_updated['location']=='ELMHURST']
new_CORONA['location'] = new_CORONA.location.str.replace('ELMHURST','CORONA')
new_FOREST_HILLS = df_median_DISTRICTS_updated[df_median_DISTRICTS_updated['location']=='REGO PARK']
new_FOREST_HILLS['location'] = new_FOREST_HILLS.location.str.replace('REGO PARK','FOREST HILLS')
new_WOOD_SIDE = df_median_DISTRICTS_updated[df_median_DISTRICTS_updated['location']=='Sunnyside']
new_WOOD_SIDE['location'] = new_WOOD_SIDE.location.str.replace('Sunnyside','WOODSIDE')
df_median_DISTRICTS_updated = pd.concat([df_median_DISTRICTS_updated, new_CORONA, new_FOREST_HILLS, new_WOOD_SIDE])
df_median_DISTRICTS_updated

,location,all_households,families,families_with_children,families_without_children
0,ASTORIA,"$79,180","$88,662","$75,972","$92,755"
1,BATTERY PARK/TRIBECA,"$162,092","$230,219","$250,001","$204,568"
2,BAY RIDGE,"$76,569","$89,345","$91,935","$87,713"
3,BAYSIDE,"$92,682","$108,371","$120,639","$102,316"
4,BEDFORD PARK,"$41,336","$47,784","$40,863","$56,260"
...,...,...,...,...,...
57,WILLIAMSBURG/GREENPOINT,"$98,284","$81,507","$74,327","$88,784"
58,WOODHAVEN,"$76,978","$87,282","$91,903","$85,907"
21,CORONA,"$67,143","$67,886","$58,182","$78,892"
40,FOREST HILLS,"$89,160","$120,120","$134,289","$105,599"


In [9]:
df_median_ZipCodes = pd.DataFrame(df, columns=df[1])[70:].reset_index(drop=True)
df_median_ZipCodes['Rank / Location'] = [df_median_ZipCodes['Rank / Location'][i][-5:] for i in range(len(df_median_ZipCodes))]
df_median_ZipCodes.columns = defined_columns
df_median_ZipCodes

,location,all_households,families,families_with_children,families_without_children
0,10001,"$92,840","$122,776","$106,452","$132,545"
1,10002,"$36,982","$40,045","$42,054","$38,639"
2,10003,"$118,161","$183,787","$250,001","$153,480"
3,10004,"$190,223","$205,202","$244,000","$204,130"
4,10005,"$189,702","$241,094","$250,001","$202,042"
...,...,...,...,...,...
176,11691,"$50,267","$61,360","$60,325","$64,441"
177,11692,"$46,819","$50,516","$42,614","$57,188"
178,11693,"$60,569","$66,250","$62,188","$67,741"
179,11694,"$84,485","$111,046","$120,943","$107,721"


In [10]:
df = pd.concat([df_median_BOROUGHS, df_median_DISTRICTS, df_median_ZipCodes])
df

,location,all_households,families,families_with_children,families_without_children
0,Bronx,"$41,432","$50,835","$41,129","$61,248"
1,Brooklyn,"$66,937","$74,422","$66,936","$79,400"
2,Manhattan,"$93,651","$126,690","$140,841","$121,669"
3,Queens,"$73,696","$82,534","$75,501","$86,501"
4,Staten Island,"$89,821","$105,438","$104,641","$106,015"
...,...,...,...,...,...
176,11691,"$50,267","$61,360","$60,325","$64,441"
177,11692,"$46,819","$50,516","$42,614","$57,188"
178,11693,"$60,569","$66,250","$62,188","$67,741"
179,11694,"$84,485","$111,046","$120,943","$107,721"


In [11]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 'ia-final-project-bucket/'#specify location of s3:/{my-bucket}/
filenames = f"{pathname}nyc_BOROUGHS_median_income_info.csv" #name of the filepath and csv file

#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df_median_BOROUGHS.to_csv(None, index=False).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames))

Successfull uploaded file to location:ia-final-project-bucket/nyc_BOROUGHS_median_income_info.csv


In [12]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 'ia-final-project-bucket/'#specify location of s3:/{my-bucket}/
filenames = f"{pathname}nyc_ZipCodes_median_income_info.csv" #name of the filepath and csv file

#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df_median_ZipCodes.to_csv(None, index=False).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames))

Successfull uploaded file to location:ia-final-project-bucket/nyc_ZipCodes_median_income_info.csv


In [13]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 'ia-final-project-bucket/'#specify location of s3:/{my-bucket}/
filenames = f"{pathname}nyc_DISTRICTS_median_income_info.csv" #name of the filepath and csv file

#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df_median_DISTRICTS_updated.to_csv(None, index=False).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames))

Successfull uploaded file to location:ia-final-project-bucket/nyc_DISTRICTS_median_income_info.csv
